# Imports

In [9]:
import xmltodict

# XML Parser

In [12]:
xml_file = 'config_tigerLake.xml'
with open(xml_file, 'r') as file:
    xml_data = file.read()

xml_dict = xmltodict.parse(xml_data)

In [15]:
xml_dict['Configuration']['System'].keys()

dict_keys(['MainMemory', 'CacheBusLatency', 'GlobalClock', 'Core', 'SharedCaches', 'Interconnect', 'NOC', 'BUS', 'MainMemoryController'])